In [1]:
# These are helpful routines that will assist in building this book. 
# You should run this block before anything else. There is no output expected.
from astrodynamicsbook.bookhelpers import *
loadLatexPreamble()

# This is only here to create the navigation link:
genPrevLink()

$
\def\bs{\boldsymbol}
\def\mf{\mathbf}
\def\mb{\mathbb}
\def\mc{\mathcal}
\def\rfr{\mathcal}
\def\grad{{\rm grad}}
\def\Re{{\rm Re}}
\def\Im{{\rm Im}}
\def\und{\underline}
\def\ovl{\overline}
\def\unb{\underbrace}
\def\Log{\mbox{Log}}
\def\bfomega{\bs \omega}
\def\bfalpha{\bs \alpha}
\def\da{\triangleq}
\newcommand{\leftexp}[2]{{\vphantom{#2}}^{#1}\!{#2}}
\newcommand{\leftsub}[2]{{\vphantom{#2}}_{#1}\!{#2}}
\newcommand{\omegarot}[2]{{\leftexp{\mathcal{#1}}{\boldsymbol{\omega}}^{\mathcal{#2}}}}
\newcommand{\alpharot}[2]{{\leftexp{\mathcal{#1}}{\boldsymbol{\alpha}}^{\mathcal{#2}}}}
\newcommand{\framerot}[2]{{\leftexp{\mathcal{#1}}{C}^{\mathcal{#2}}}}
\newcommand{\dframerot}[2]{{\vphantom{\dot{C}}^{\mathcal{#1}}\!{\dot{C}^{\mathcal{#2}}}}}
\newcommand{\bdot}[1]{\dot{\mathbf{#1}}}
\newcommand{\bhat}[1]{\hat{\mathbf{#1}}}
\newcommand{\mbhat}[1]{\hat{\mathbb{#1}}}
\def\iwb{\omegarot{I}{B}}
\def\iab{\alpharot{I}{B}}
\def\icb{\framerot{I}{B}}
\def\dif{\mathop{}\!\mathrm{d}}
\newcommand{\intd}[1]{\dif#1}
\newcommand{\od}[3][]{{ \frac{\dif{^{#1}}#2}{\dif{#3^{#1}}} }}			
\newcommand{\pd}[3][]{{ \frac{\partial{^{#1}}#2}{\partial{#3^{#1}}} }}	 
\newcommand{\md}[6]{{  \frac{\partial{^{#2}}#1}{\partial{#3^{#4}}\partial{#5^{#6}}} }}
\newcommand{\fddt}[2][]{{  \leftexp{\mathcal{#2}}{\frac{\dif{#1}}{\dif{t}}}  }}
\newcommand{\fdddt}[2][]{{  \leftexp{\mathcal{#2}}{\frac{\dif{^{2}#1}}{\dif{t^2}}}  }}
\newcommand{\ddt}[1][]{\fddt[#1]{I}}
$


# [Previous](<15-Keplers Time Equation.ipynb>)

# Inverting Kepler's Time Equation

Recall our impetus for deriving the time equation in the first place:  We wished to write our conic section solution explicitly as a function of time, and found that we couldn't.  We now have a daisy-chain of equations mapping from $\nu$ to $E$ to $M$ and $M$ is directly proportional to time, so, presumably all that's left to do is to invert the time equation.

And...we can't. It's a transcendental expression with no closed form solution. So, that was all a giant waste of time, huh?

Well, note quite.  While impossible to invert analytically, the time equation is incredibly straight-forward to invert *numerically* using any number of well-established algorithms. Let's explore.

# Newton-Raphson Iteration

**Newton-Raphson** iteration (sometimes known as Newton's method, and named in honor - surprise - of Newton and Joseph Raphson, whose greatest claim to fame is how shockingly little we know about his life) is one of the most basic and powerful methods for root-finding (that is, approximating the zeroes of some function). The method goes as follows:

<div class="alert alert-block alert-info">
    <b>Newton-Raphson Iteration</b>
    <ol>
        <li>Given a real-valued function $f(x) = 0, \, x\in \mathbb{R}$, its derivative with respect to $x$:
        $$f'(x) = \frac{\intd{f}}{\intd{x}}$$
        and some starting guess $x_0$</li>
        <li> Compute the next guess (iterant) as:
            $$x_{n+1} = x_n - \frac{f(x_n)}{f'(x_n)}$$</li>
        <li> Continue updating the solution until converged, that is until:
            $$\vert x_{n+1} - x_n\vert < \epsilon$$
            for some small value $\epsilon$</li>
    </ol>
    </div>
    
## Newton-Raphson Iteration of the Kepler Time Equation

We can easily apply this method to the time equation by first rewriting it to a form that equals zero:
$$M - (E - e\sin(E)) = 0$$
Differentiating with respect to $E$ yields $e\cos(E) - 1$ and so our iteration is given by:
$$E_{n+1} = E_n - \frac{M - E_n + e\sin(E_n)}{e\cos(E_n) - 1}$$

The only remaining questions are, what should we use as an initial guess, and what should we use as a convergence criterion? Let's consider the latter question first.

# Computer Representations of Real Numbers

This feels like a huge aside, but is actually a crucial discussion to have before embarking on any numerical adventures.  The fact is, we will be doing our math on a computer, and it is important to realize that computers have many different ways of encoding numbers, and most are highly non-intuitive and can lead to some pretty surprising behavior. For a start, consider the following:

$$1 + 1\times 10^{-16} - 1$$
$$1 + 1\times 10^{-14} - 1$$

We know that these two expressions must evaluate to $1\times 10^{-16}$ and $1\times 10^{-14}$, respectively.  But does *your computer* know this?  Let's find out:

In [9]:
1 + 1e-16 - 1

In [10]:
1 + 1e-14 - 1

What?!?  This is madness.  Not only did neither operation produce the correct answer, but they didn't even fail in the same way. The good news is that this was fully expected and deterministic behavior.  The *better* news is that we get to learn about something new.

Say that you wish to encode an arbitrary real number in a computer.  At the most basic level, computers store things in registers as collections of bits, which are binary 0-1 values.

<div class="alert alert-block alert-danger">
    We're going to be simplifying <i>hugely</i> here so, CS and EE majors: shut your ears.
    </div>

Thus, the amount of 